# Prognostika
- Comprehensive ML to predict hard drive failures.

# Data Analysis and Case Study
- This notebook is written to keep track of stastical data and condcut case study.

In [ ]:
import pandas as pd
import numpy as np
import math
import pickle
import matplotlib.pyplot as plt
import scipy
import scipy.stats
import dask.dataframe as dd
from collections import Counter
import os
from tqdm.notebook import tqdm
import glob
import seaborn as sns
import warnings
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import matplotlib.colors
warnings.filterwarnings('ignore')
pd.set_option('display.max_rows', None)  # Shows all rows
pd.set_option('display.max_columns', None)  # Shows all columns
pd.set_option('display.width', None)  # Uses the maximum width to display each column
pd.set_option('display.max_colwidth', None)  # Shows full length of the data in columns

- Generate data based on given parameters:


In [ ]:
# Define parameters for years and quarters
years = ['2016']
quarters = ['Q1']  # Define which quarters to process
required_features = ['serial_number', 'model', 'failure', 'date']
all_data = []

# Get the script directory
script_dir = os.path.dirname(os.path.abspath(''))

# Join quarters to form the part of the file name
quarter_part = '_'.join(quarters) if quarters else 'full_year'

# Construct the pickle file name
pickle_file = os.path.join(script_dir, 'output', f"{'_'.join(years)}_{quarter_part}.pkl")

# Quarter date ranges
quarter_ranges = {
    'Q1': ('01-01', '03-31'),
    'Q2': ('04-01', '06-30'),
    'Q3': ('07-01', '09-30'),
    'Q4': ('10-01', '12-31')
}

# Check if the pickle file already exists
if os.path.exists(pickle_file):
    print(f"Loading data from {pickle_file}...")
    with open(pickle_file, 'rb') as f:
        df = pickle.load(f)
    print("Data loaded successfully from pickle file.")
else:
    # Loop over each year
    for y in tqdm(years, desc="Analyzing years"):
        # Construct the path to get all files for the year
        path = os.path.join(script_dir, 'HDD_dataset', y, '*.csv')
        files = glob.glob(path)
        
        # Filter files based on quarters
        filtered_files = []
        for q in quarters:
            start_date = f"{y}-{quarter_ranges[q][0]}"
            end_date = f"{y}-{quarter_ranges[q][1]}"
            filtered_files += [f for f in files if start_date <= os.path.basename(f)[:10] <= end_date]
        
        # Loop over each file that falls within the selected quarters
        for f in tqdm(filtered_files, desc=f"Analyzing files in {y} {', '.join(quarters)}"):
            try:
                # Read the first row to determine the columns
                temp_df = pd.read_csv(f, nrows=1)
                all_columns = temp_df.columns

                # Identify columns containing the keyword 'raw'
                raw_columns = [col for col in all_columns if 'raw' in col]

                # Combine required features with raw columns
                features = required_features + raw_columns

                # Read the CSV file with specified features
                data = pd.read_csv(f, usecols=features, parse_dates=['date'], low_memory=False)
                data['failure'] = data['failure'].astype('int')
                all_data.append(data)
            except Exception as e:
                print(f"Error reading file {f}: {e}")

    # Combine all data into a single DataFrame if there is any data
    if all_data:
        df = pd.concat(all_data, ignore_index=True)
        df.set_index(['serial_number', 'date'], inplace=True)
        df.sort_index(inplace=True)
        
        # Save the DataFrame to a pickle file
        os.makedirs(os.path.join(script_dir, 'output'), exist_ok=True)
        with open(pickle_file, 'wb') as f:
            pickle.dump(df, f)
        print(f"Data concatenated and saved to {pickle_file} successfully.")
    else:
        print("No data to concatenate.")

# Backup the DataFrame
backup_df = df.copy()

# Display the DataFrame
print('Data concatenated successfully.')

In [ ]:
df = backup_df.copy() # Restore the original DataFrame

- Split into failure vs non-failure drives serial number

In [ ]:
# Split the DataFrame into failures and non-failures
failure_df = df[df['failure'] == 1]
non_failure_df = df[df['failure'] == 0]

print(f"Data split into failures and non-failures.")
print(f"Failures: {failure_df.shape[0]} records.")
print(f"Non-failures: {non_failure_df.shape[0]} records.")

# Manufacturer-level and Model-level analysis
- The values of SMART stats and whether or not they are reported varies from vendor to vendor. Furthermore, simply including vendor as a feature may or may not work for all kinds of prediction models. Therefore, it may be a good idea to analyze the data in a vendor specific way.

In [ ]:
# You can first not run his.
# Total number of records in the dataset
total_records = df.shape[0]

# Splitting the DataFrame by manufacturer and calculating percentages
seagate_df = df[df["model"].str.startswith("S")]
seagate_percentage = (seagate_df.shape[0] / total_records) * 100
print(f"Seagate records: {seagate_df.shape}, {seagate_percentage:.2f}% of total")

hgst_df = df[df["model"].str.startswith("HG")]
hgst_percentage = (hgst_df.shape[0] / total_records) * 100
print(f"HGST records: {hgst_df.shape}, {hgst_percentage:.2f}% of total")

toshiba_df = df[df["model"].str.startswith("T")]
toshiba_percentage = (toshiba_df.shape[0] / total_records) * 100
print(f"Toshiba records: {toshiba_df.shape}, {toshiba_percentage:.2f}% of total")

wdc_df = df[df["model"].str.startswith("W")]
wdc_percentage = (wdc_df.shape[0] / total_records) * 100
print(f"WDC records: {wdc_df.shape}, {wdc_percentage:.2f}% of total")

hitachi_df = df[df["model"].str.startswith("Hi")]
hitachi_percentage = (hitachi_df.shape[0] / total_records) * 100
print(f"Hitachi records: {hitachi_df.shape}, {hitachi_percentage:.2f}% of total")


# Define the data
labels = ['Seagate', 'HGST', 'Toshiba', 'WDC', 'Hitachi']
sizes = [seagate_percentage, hgst_percentage, toshiba_percentage, wdc_percentage, hitachi_percentage]

# Create a pie chart
fig1, ax1 = plt.subplots()
ax1.pie(sizes, labels=labels, autopct='%1.1f%%', startangle=90)

# Equal aspect ratio ensures that pie is drawn as a circle
ax1.axis('equal')  

plt.show()

- Show some statistical data

In [ ]:
# Define a function to assign vendor based on model prefix
def assign_vendor(model):
    if model.startswith("S"):
        return "Seagate"
    elif model.startswith("HG"):
        return "HGST"
    elif model.startswith("T"):
        return "Toshiba"
    elif model.startswith("W"):
        return "WDC"
    elif model.startswith("Hi"):
        return "Hitachi"
    else:
        return "Unknown"
    

# Calculate total failures and non-failures for each model
model_failures = failure_df['model'].value_counts()
model_non_failures = non_failure_df['model'].value_counts()

# Calculate total counts and percentages
total_counts = df['model'].value_counts()
failure_percentages = (model_failures / total_counts * 100).fillna(0)  # fill NaN with 0 where no failures are recorded

# Sort models by total failure count in descending order
sorted_models = model_failures.sort_values(ascending=False)

# Prepare data for display
data = []
for model in sorted_models.index:
    fail_count = sorted_models[model]
    non_fail_count = model_non_failures.get(model, 0)
    total_count = total_counts[model]
    total_percent = (total_count / total_counts.sum()) * 100
    fail_percent = (fail_count / total_count) * 100 if total_count else 0

    data.append({
        "Model": model,
        "Failure count": fail_count,
        "Non-failure count": non_fail_count,
        "Total count": total_count,
        "Total percent": total_percent,  # Keeping as float
        "Fail percent": fail_percent,   # Keeping as float
        "Vendor": assign_vendor(model)
    })

# Convert the list of dictionaries into a DataFrame
report_df = pd.DataFrame(data)

# Set display options (optional, for full display in Jupyter Notebook)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

# Display the DataFrame
display(report_df)

- Plot top 10 data:

In [ ]:
# Extract top 10 models by failure percentage
top10_fail_percent = report_df.nlargest(10, 'Fail percent')

plt.figure(figsize=(12, 8))
barplot = sns.barplot(x='Fail percent', y='Model', data=top10_fail_percent, palette='viridis')
plt.title('Top 10 Models by Failure Percentage')
plt.xlabel('Failure Percentage')
plt.ylabel('Model')

# Adding annotations to each bar
for p in barplot.patches:
    width = p.get_width()    # Get the width of the bar
    plt.text(x=width + 0.5,  # Place text at 0.5 units to the right of the bar
             y=p.get_y() + p.get_height() / 2,  # Vertically align text in the middle of the bar
             s='{:.2f}%'.format(width),  # The text to display
             va='center')

plt.show()

# Extract top 10 models by total failure count
top10_fail_count = report_df.nlargest(10, 'Failure count')

plt.figure(figsize=(12, 8))
barplot = sns.barplot(x='Failure count', y='Model', data=top10_fail_count, palette='magma')
plt.title('Top 10 Models by Total Failure Count')
plt.xlabel('Total Failure Count')
plt.ylabel('Model')

# Adding annotations to each bar
for p in barplot.patches:
    width = p.get_width()    # Get the width of the bar
    plt.text(x=width + 3,    # Place text at 3 units to the right of the bar
             y=p.get_y() + p.get_height() / 2,  # Vertically align text in the middle of the bar
             s=int(width),  # The text to display, converted to int for clean display
             va='center')

plt.show()


- Analysis by manufacture instead of extact model:

In [ ]:
# Calculate total number of records in the dataset
total_records = df.shape[0]

# Splitting the DataFrame by manufacturer and calculating percentages
vendors = ["Seagate", "HGST", "Toshiba", "WDC", "Hitachi"]
vendor_data = {}

for vendor in vendors:
    vendor_df = df[df["model"].str.startswith(vendor[0])]
    vendor_percentage = (vendor_df.shape[0] / total_records) * 100
    vendor_data[vendor] = {
        "records": vendor_df.shape[0],
        "percentage": vendor_percentage
    }
    print(f"{vendor} records: {vendor_df.shape}, {vendor_percentage:.2f}% of total")

# Calculate total failures and non-failures for each model
model_failures = failure_df['model'].value_counts()
model_non_failures = non_failure_df['model'].value_counts()

# Calculate total counts by vendor
vendor_failures = {}
vendor_non_failures = {}

for vendor in vendors:
    vendor_failures[vendor] = model_failures[model_failures.index.str.startswith(vendor[0])].sum()
    vendor_non_failures[vendor] = model_non_failures[model_non_failures.index.str.startswith(vendor[0])].sum()

# Prepare data for display by vendor
vendor_report = []

for vendor in vendors:
    fail_count = vendor_failures[vendor]
    non_fail_count = vendor_non_failures[vendor]
    total_count = vendor_data[vendor]["records"]
    total_percent = vendor_data[vendor]["percentage"]
    fail_percent = (fail_count / total_count) * 100 if total_count else 0

    vendor_report.append({
        "Vendor": vendor,
        "Failure count": fail_count,
        "Non-failure count": non_fail_count,
        "Total count": total_count,
        "Total percent": total_percent,  # Keeping as float
        "Fail percent": fail_percent   # Keeping as float
    })
print("Notes: In failure percentage, 1.00 means 1%, so 0.007687 means 0.007687% failure rate.")
# Convert the list of dictionaries into a DataFrame
vendor_report_df = pd.DataFrame(vendor_report)

# Convert the list of dictionaries into a DataFrame
vendor_report_df = pd.DataFrame(vendor_report)

# Print the note
print("Notes: In failure percentage, 1.00 means 1%, so 0.007687 means 0.007687% failure rate.")

# Display the DataFrame
display(vendor_report_df)

# Plotting failure percentages by vendor
plt.figure(figsize=(12, 8))
barplot = sns.barplot(x='Fail percent', y='Vendor', data=vendor_report_df, palette='viridis')
plt.title('Failure Percentage by Vendor')
plt.xlabel('Failure Percentage')
plt.ylabel('Vendor')

# Adding annotations to each bar
for p in barplot.patches:
    width = p.get_width()
    plt.text(x=width + 0.5,
             y=p.get_y() + p.get_height() / 2,
             s='{:.5f}%'.format(width),
             va='center')

plt.show()

# Plotting total failure counts by vendor
plt.figure(figsize=(12, 8))
barplot = sns.barplot(x='Failure count', y='Vendor', data=vendor_report_df, palette='magma')
plt.title('Total Failure Count by Vendor')
plt.xlabel('Total Failure Count')
plt.ylabel('Vendor')

# Adding annotations to each bar
for p in barplot.patches:
    width = p.get_width()
    plt.text(x=width + 3,
             y=p.get_y() + p.get_height() / 2,
             s=int(width),
             va='center')

plt.show()
print(df.columns)

## Analyze on SMART attributes
### Analyze on critical Stats
- Backblaze mentions five stats as better predictors than others. These are 5, 187, 188, 197, 198.They also provide some analysis using these features.
- Show some nan stats

In [ ]:
CRITICAL_STATS = [
    1,
    5,
    7,
    10,
    184,
    187,
    188,
    189,
    190,
    193,
    194,
    196,
    197,
    198,
    201,
    240,
    241,
    242,
]

# Create a copy of the DataFrame
df_smart_stats = df.copy()

# List of critical stats columns to check for NaNs
critical_columns = [f"smart_{stat}_raw" for stat in CRITICAL_STATS]

# Ensure the columns exist in the DataFrame
existing_columns = [col for col in critical_columns if col in df_smart_stats.columns]

# Calculate the number of NaN values in each critical column
nan_counts = df_smart_stats[existing_columns].isna().sum()

# Calculate the percentage of NaN values
total_rows = df_smart_stats.shape[0]
nan_percentages = (nan_counts / total_rows) * 100

# Combine counts and percentages into a DataFrame
nan_stats = pd.DataFrame({
    'NaN Count': nan_counts,
    'NaN Percentage': nan_percentages
})

# Display the results
print("Number and percentage of NaN values in critical columns:")
display(nan_stats)


- NaN Counts based on manufacturer.

In [ ]:
# Define the list of critical SMART attributes
CRITICAL_STATS = [
    1, 5, 7, 10, 184, 187, 188, 189, 190, 193, 194, 196, 197, 198, 201, 240, 241, 242,
]

# Generate column names for critical SMART attributes
critical_columns = [f"smart_{stat}_raw" for stat in CRITICAL_STATS]

# Ensure the columns exist in the DataFrame
existing_columns = [col for col in critical_columns if col in df.columns]

# Generate DataFrames based on manufacturers
seagate_df = df[df["model"].str.startswith("S")]
hgst_df = df[df["model"].str.startswith("HG")]
toshiba_df = df[df["model"].str.startswith("T")]
wdc_df = df[df["model"].str.startswith("W")]
hitachi_df = df[df["model"].str.startswith("Hi")]

# List of manufacturer DataFrames
vendor_dfs = {
    "Seagate": seagate_df,
    "HGST": hgst_df,
    "Toshiba": toshiba_df,
    "WDC": wdc_df,
    "Hitachi": hitachi_df
}

# Function to calculate NaN counts and percentages
def calculate_nan_stats(df, existing_columns):
    total_rows = df.shape[0]
    nan_counts = df[existing_columns].isna().sum()
    nan_percentages = (nan_counts / total_rows) * 100
    return pd.DataFrame({
        'NaN Count': nan_counts,
        'NaN Percentage': nan_percentages
    })

# Calculate and print NaN stats for each vendor
for vendor, vendor_df in vendor_dfs.items():
    print(f"\nNaN stats for {vendor}:")
    nan_stats = calculate_nan_stats(vendor_df, existing_columns)
    display(nan_stats)

- Let's drop some useless columns to better reflect our data.

In [ ]:
df = df.loc[df['failure'].isin([0, 1])]
# Drop columns that contain the keyword 'normalized'
df = df.drop(columns=[col for col in df.columns if 'normalized' in col])

# Dictionary to store DataFrames post each threshold
dataframes_post_threshold = {}

# Calculate the number of original columns
original_column_count = df.shape[1]

# Dictionary to hold columns post-threshold application
columns_post_threshold = {}

# Drop columns where more than 50% of the values are NaN
threshold_50 = len(df) * 0.5
df_50 = df.dropna(thresh=threshold_50, axis=1)
columns_post_threshold['50%'] = df_50.columns

# Count and print results for the 50% threshold
columns_dropped_50 = original_column_count - len(df_50.columns)
print(f"After 50% threshold:")
print(f"Total columns retained: {len(df_50.columns)}")
print(f"Total columns dropped: {columns_dropped_50}")
print(f"Ratio of columns dropped: {columns_dropped_50 / original_column_count:.2f}\n")

# Process for different thresholds (25%, 10%, 1%)
thresholds = [0.25, 0.10, 0.01]
for threshold in thresholds:
    threshold_value = len(df) * (1 - threshold)
    new_df = df.dropna(thresh=threshold_value, axis=1)
    columns_post_threshold[f'{threshold*100:.0f}%'] = new_df.columns
    dataframes_post_threshold[f'{threshold*100:.0f}%'] = new_df  # Save each DataFrame
    
    # Calculate drops and print results for each threshold
    columns_dropped = original_column_count - len(new_df.columns)
    print(f"After {threshold*100:.0f}% threshold:")
    print(f"Total columns retained: {len(new_df.columns)}")
    print(f"Total columns dropped: {columns_dropped}")
    print(f"Ratio of columns dropped: {columns_dropped / original_column_count:.2f}\n")

# Print columns retained for each threshold
for key, columns in columns_post_threshold.items():
    print(f"Columns retained at {key} threshold:")
    print(list(columns))
    print()  # Add an empty line for better separation

# Optionally, access and use the DataFrame retained at the 1% threshold
one_percent_df = dataframes_post_threshold['1%']

In [ ]:
new_df = df_50.copy()

In [ ]:
# method = 'linear'
# interp_dfs = []
# total_interpolated = 0

# # Specify the columns to interpolate
# columns_to_interpolate = ['failure']  # Add any other columns that you want to interpolate

# for serial_num, inner_df in new_df.groupby(level=0):
#     inner_df = inner_df.droplevel(level=0).asfreq('D')

#     # Count the number of NaN values before interpolation
#     before_interpolation = inner_df[columns_to_interpolate].isna().sum().sum()
    
#     # Interpolate only the specified columns
#     inner_df[columns_to_interpolate] = inner_df[columns_to_interpolate].interpolate(
#         method=method, limit_direction='both', axis=0)
    
#     # Apply threshold and round
#     threshold = 1e-10
#     inner_df[columns_to_interpolate] = inner_df[columns_to_interpolate].applymap(
#         lambda x: 0 if np.abs(x) < threshold else x).round(0)

#     # Count the number of NaN values after interpolation and rounding
#     after_interpolation = inner_df[columns_to_interpolate].isna().sum().sum()
    
#     # Update the total number of interpolated values
#     total_interpolated += before_interpolation - after_interpolation
    
#     # Forward fill and backward fill to handle any remaining NaN values
#     inner_df[columns_to_interpolate] = inner_df[columns_to_interpolate].fillna(method='ffill').fillna(method='bfill')
    
#     # Add the serial number column and reset index
#     inner_df['serial_number'] = serial_num
#     inner_df = inner_df.reset_index()
    
#     # Append the processed DataFrame to the list
#     interp_dfs.append(inner_df)

# # Concatenate all DataFrames
# interp_df = pd.concat(interp_dfs, axis=0)

# # Set index and sort
# new_df = interp_df.set_index(['serial_number', 'date']).sort_index()

# # Print the results
# print(f'Total interpolated values: {total_interpolated}, '
#       f'Percentage of interpolated values: {(total_interpolated / new_df.size) * 100}%')
# print(new_df.isna().sum())
# # Initialize the dictionary to store p-values
# dict1 = {}

# for feature in new_df.columns:
#     if 'raw' in feature:
#         print(f'Distribution of {feature}:')
#         print(new_df.groupby('failure')[feature].describe())

# # Perform statistical tests on features containing 'raw'
# for feature in new_df.columns:
#     if 'raw' in feature:
#         print(f'Feature: {feature}')

#         # Conduct t-test
#         _, p_val_ttest = scipy.stats.ttest_ind(new_df[new_df['failure'] == 0][feature],
#                                                new_df[new_df['failure'] == 1][feature], 
#                                                axis=0, nan_policy='omit')
#         print(f'T-test p-value for {feature}: {p_val_ttest:.6f}')

#         # Conduct Mann-Whitney U test
#         _, p_val_mannu = scipy.stats.mannwhitneyu(new_df[new_df['failure'] == 0][feature], 
#                                                   new_df[new_df['failure'] == 1][feature], 
#                                                   alternative='two-sided')
#         print(f'Mann-Whitney U test p-value for {feature}: {p_val_mannu:.6f}')

#         # Store p-values in the dictionary
#         dict1[feature] = {'T-test': p_val_ttest, 'Mann-Whitney U': p_val_mannu}

# # Sort features based on the p-values of t-tests
# sorted_features_t = {k: v for k, v in sorted(dict1.items(), key=lambda item: item[1]['T-test'])}
# sorted_features_mannu = {k: v for k, v in sorted(dict1.items(), key=lambda item: item[1]['Mann-Whitney U'])}

# # Convert dictionary to DataFrame and display all features
# features_df = pd.DataFrame(sorted_features_t).transpose()
# print('Features sorted by T-test p-values:')
# print(features_df)

# features_df = pd.DataFrame(sorted_features_mannu).transpose()
# print('Features sorted by Mann-Whitney U p-values:')
# print(features_df)


- Now let's do some basic statical things: (dropnas)

In [ ]:
# Assuming 'df' is your original DataFrame
total_rows_before = len(new_df)

# Drop rows where any column has NaN values
clean_df = new_df.dropna()

# Calculate the total number of rows after dropping
total_rows_after = len(clean_df)

# Calculate the percentage of rows dropped
percentage_dropped = ((total_rows_before - total_rows_after) / total_rows_before) * 100

print(f"Percentage of rows dropped: {percentage_dropped:.2f}%")

print(clean_df.isna().sum())
# Initialize the dictionary to store p-values
dict1 = {}

for feature in clean_df.columns:
    if 'raw' in feature:
        print(f'Distribution of {feature}:')
        print(new_df.groupby('failure')[feature].describe())

# Perform statistical tests on features containing 'raw'
for feature in clean_df.columns:
    if 'raw' in feature:
        print(f'Feature: {feature}')

        # Conduct t-test
        _, p_val_ttest = scipy.stats.ttest_ind(clean_df[clean_df['failure'] == 0][feature],
                                               clean_df[clean_df['failure'] == 1][feature], 
                                               axis=0, nan_policy='omit')
        print(f'T-test p-value for {feature}: {p_val_ttest:.6f}')

        # Conduct Mann-Whitney U test
        _, p_val_mannu = scipy.stats.mannwhitneyu(clean_df[clean_df['failure'] == 0][feature], 
                                                  clean_df[clean_df['failure'] == 1][feature], 
                                                  alternative='two-sided')
        print(f'Mann-Whitney U test p-value for {feature}: {p_val_mannu:.6f}')

        # Store p-values in the dictionary
        dict1[feature] = {'T-test': p_val_ttest, 'Mann-Whitney U': p_val_mannu}

# Sort features based on the p-values of t-tests
sorted_features_t = {k: v for k, v in sorted(dict1.items(), key=lambda item: item[1]['T-test'])}
sorted_features_mannu = {k: v for k, v in sorted(dict1.items(), key=lambda item: item[1]['Mann-Whitney U'])}

# Convert dictionary to DataFrame and display all features
features_df = pd.DataFrame(sorted_features_t).transpose()
print('Features sorted by T-test p-values:')
print(features_df)

features_df = pd.DataFrame(sorted_features_mannu).transpose()
print('Features sorted by Mann-Whitney U p-values:')
print(features_df)

In [ ]:
# Number of rows to use for plotting - sampling is done if the number of rows is too much to keep in memory
num_rows_to_sample = 5000000
hist_cols = new_df.columns  # Removed the filtering on NaN percentage
len_df = len(new_df)

for col in hist_cols:
    # Get all the values for this column
    if num_rows_to_sample < len_df:
        data = new_df[col].sample(frac=num_rows_to_sample / len_df)
    else:
        data = new_df[col]

    # Print the number of NaN values and plot only the non-null values
    print(
        data.isna().sum(),
        "out of",
        data.shape[0],
        "are NaN values. These are not shown on the graph below",
    )
    
    # If no non-NaN data is available, skip plotting
    if data.notna().sum() > 0:
        sns.histplot(data.dropna(), kde=True)  # Using histplot for histograms with KDE
        plt.title(f'Histogram of {col}')  # Optional: Add title for clarity
        plt.xlabel(f'Values of {col}')  # Optional: Label the x-axis
        plt.ylabel('Frequency')  # Optional: Label the y-axis
        plt.show()
    else:
        print(f"No non-NaN data available in {col}. Skipping histogram.")

In [ ]:
# Define correlation columns: including 'failure', 'raw'
corr_cols = ["failure"] + [col for col in one_percent_df.columns if 'raw' in col]
# Downsampling serial numbers from both failed and working sets
downsampled_sers = (
    failure_df.sample(n=min(500, len(failure_df))).values.tolist() +
    non_failure_df.sample(n=min(75000, len(non_failure_df))).values.tolist()
)
print(failure_df)
print(non_failure_df)

# Reset the index to turn 'serial_number' into a column
one_percent_df_reset = one_percent_df.reset_index()
one_percent_df_reset.drop(columns=["model"])
print(one_percent_df_reset.columns)
corr_mat = one_percent_df_reset[one_percent_df_reset["serial_number"].isin(downsampled_sers)][corr_cols].corr()
print(corr_mat)

# Visualization with a heatmap
fig, ax = plt.subplots(figsize=(10, 10))
sns.heatmap(
    corr_mat,
    ax=ax,
    vmin=-1,
    vmax=1,
    square=True,
    linewidths=0.5,
    cmap="RdBu",
    cbar_kws={"shrink": 0.5},
)
plt.show()



In [ ]:
# List of manufacturer DataFrame names for dynamic access
manufacturers_data = {
    'Seagate': seagate_df,
    'WDC': wdc_df,
    'HGST': hgst_df,
    'Hitachi': hitachi_df,
    'Toshiba': toshiba_df
}

NAN_PERCENT_THRESHOLD = 0.01  # Define your NAN percent threshold

# Calculate the percentage of NaNs for each manufacturer's DataFrame and store it
manufacturers_nan_ct = {name: data.isna().mean() for name, data in manufacturers_data.items()}

# Loop over each manufacturer
for name, data_df in manufacturers_data.items():
    nan_ct = manufacturers_nan_ct[name]

    # Determine the correlation columns based on the NAN percent threshold
    corr_cols = ["failure"] + list(
        nan_ct[nan_ct < NAN_PERCENT_THRESHOLD].index
    )

    # Check if downsampling serial numbers are needed
    if 'downsampled_sers' in locals():
        data_df = data_df[data_df["serial_number"].isin(downsampled_sers)]

    # Compute the correlation matrix
    corr_mat = data_df[corr_cols].corr()

    # Plotting the heatmap
    fig, ax = plt.subplots(figsize=(10, 10))
    sns.heatmap(
        corr_mat,
        ax=ax,
        vmin=-1,
        vmax=1,
        square=True,
        linewidths=0.5,
        cmap="RdBu",
        cbar_kws={"shrink": 0.5},
    )
    ax.set_title(f'Correlation Matrix for {name} Drives')
    plt.show()


In [ ]:
# TODO: Might be better to call compute to get the combined data of all serials in subset
# as opposed to calling it for earch serial number in for loop

# NOTE: running this cell will take a VERY long time (~1hr on intel i7 w/ 16GB ram)
# adjust NUM_DRIVES_TO_SAMPLE to select a small subset to use for plotting
NUM_DRIVES_TO_SAMPLE = 50

# plots for smart stat 5
# cols_to_plot = ['smart_5_raw', 'smart_7_raw']

cols_to_plot = ['smart_' + str(attr) + '_raw' for attr in CRITICAL_STATS]
cols_to_plot.remove("smart_201_raw")  # too many nans
cols_to_plot.remove("smart_201_normalized")  # too many nans

# one figure per smart stat
figs = [plt.figure(i, figsize=(10, 10)) for i in range(len(cols_to_plot))]
axes = [f.add_subplot(111) for f in figs]
for colname, ax in zip(cols_to_plot, axes):
    ax.set_title(
        "{} vs Time for {} Failed Drives".format(colname, NUM_DRIVES_TO_SAMPLE)
    )
    ax.set_xlabel("Number of Days")
    ax.set_ylabel(colname)

# keep track of what hard drives were used to generate the data. NOTE: only the first 16 chars of ser will be saved
failed_ser_subset = np.empty(shape=(NUM_DRIVES_TO_SAMPLE), dtype="<S16")

# make the figures
for i, ser in enumerate(
    failure_df["serial_number"].sample(NUM_DRIVES_TO_SAMPLE, random_state=42)
):
    # log serial numbers which are being used
    failed_ser_subset[i] = ser
    print("{} / {}. Drive serial number {}".format(i + 1, NUM_DRIVES_TO_SAMPLE, ser))

    # get teh data to make the figures
    drive_data = df[df["serial_number"] == ser][cols_to_plot].compute()

    # dummy x axis data
    xvals = [i for i in range(drive_data.shape[0])]

    # make the plot
    for ax, c in zip(axes, cols_to_plot):
        ax.plot(xvals, drive_data[c])

# save the figures
for f, c in zip(figs, cols_to_plot):
    f.savefig("output/{}_failed.png".format(c))

# save the serial numbres used in figures
np.save("failed_graphs_serials", failed_ser_subset)

## PCA analysis
- For each group, which corresponds to a unique serial number (hence a unique hard drive), the mean and standard deviation of each column are computed. This helps in summarizing the time-series data into statistical features which are easier to analyze.
- The aggregated mean and standard deviation DataFrames are concatenated side by side. This results in a DataFrame where each row represents a hard drive, and each column represents a mean or standard deviation of a feature.

In [ ]:
groups = df_50.groupby("serial_number")

- For each group, which corresponds to a unique serial number (hence a unique hard drive), the mean and standard deviation of each column are computed. This helps in summarizing the time-series data into statistical features which are easier to analyze.

In [ ]:
group_means = groups.mean().compute().add_prefix("mean_")
group_stds = groups.std().compute().add_prefix("std_")
group_stats = pd.concat([group_means, group_stds], axis=1)
group_stats["failure"] = group_stats["serial_number"].isin(failure_df["serial_number"])
group_stats.head()
clean_group_stats = group_stats.dropna(how="any") # Drop rows with any NaN values

# make sure we still have enough failure drive data
print(clean_group_stats["failure"].sum(), "failed drives data retained")
clean_group_stats[clean_group_stats["failure"]].head()

In [ ]:
# scale the data and find the top principal components
scaler = StandardScaler()
pca = PCA(n_components=3, random_state=42, whiten=True).fit_transform(
    scaler.fit_transform(clean_group_stats.drop(["serial_number", "failure"], axis=1))
)

In [ ]:
# plot pca
colors = ["blue", "red"]
fig = plt.figure(figsize=(15, 15))
ax = fig.add_subplot(111, projection="3d")
ax.scatter(
    pca[:, 0],
    pca[:, 1],
    pca[:, 2],
    c=clean_group_stats["failure"],
    cmap=matplotlib.colors.ListedColormap(colors),
)